In [1]:
import xspec
from xspec import *
import numpy as np
import matplotlib.pyplot as plot1
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import os
import signal
import pandas as pd

In [2]:
a_1 = []
a_2 = []
e_p = []
indx = []
Red_chi = []

In [3]:
#Importing the data 
os.chdir("/home/abhi/Desktop/eROSITA")
data = pd.read_excel("New  Variables and Transients.xlsx")
df_SS_IB = data[data["Source Class"] == "SS_IB"]
df_IB = data[data["Source Class"] == "IB"]
df_IB_ACS = data[data["Source Class"] == "IB_ACS"]
df_YSO = data[data["Source Class"] == "YSO"]
df_CV = data[data["Source Class"] == "CV"]
df_LPV = data[data["Source Class"] == "LPV"]
df_XRB = data[data["Source Class"] == "XRB"]

#Saving their ids 
id_SS_IB = df_SS_IB["id number"].to_numpy().astype(int)
id_IB = df_IB["id number"].to_numpy().astype(int)
id_IB_ACS = df_IB_ACS["id number"].to_numpy().astype(int)
id_CV = df_CV["id number"].to_numpy().astype(int)
id_YSO = df_YSO["id number"].to_numpy().astype(int)
id_LPV = df_LPV["id number"].to_numpy().astype(int)
id_XRB = df_XRB["id number"].to_numpy().astype(int)

In [18]:
#Plotting all the energy spectra using the looping
base_dir = "/media/abhi/Data/737_spec/LPV"  
save_dir = "media/abhi/Data/737_spec/LPV"

SS_IB_data =  id_LPV

for i in SS_IB_data:
   folder = os.path.join(base_dir, str(i))
   file_path = os.path.join(folder, "spec_grp_1.pi")
   #plot_file = os.path.join(save_dir, f"spec_{i}.png")

   os.chdir(folder)  
   xspec.AllData(file_path)
   #xspec.Plot.device = f"{plot_file} /png"
   xspec.Plot.xAxis = "keV"
   xspec.Plot("data")
   #xspec.Xset.allowPrompting = False
   xspec.Fit.nIterations = 1000
    
   #Fitting model (broken Power law) and initial parameters
   model = xspec.Model("bknpower")
   xspec.AllData.ignore("**-0.2 3.0-**")

   #initiate params
   model.bknpower.PhoIndx1 = 2
   model.bknpower.BreakE = 1
   model.bknpower.PhoIndx2 = 2
   model.bknpower.norm = 1

   #Fit
   xspec.Fit.perform()
   
   # Print parameter values
   a1 = model.bknpower.PhoIndx1.values[0]

   if a1 > 0:
      xspec.AllModels.clear() 
      model = xspec.Model("powerlaw")
      xspec.AllData.ignore("**-0.2 3.0-**")
      xspec.Fit.perform()
      a1 = model.powerlaw.PhoIndex.values[0]
      a2 = 0
      Ep = 0
      red_chi = Fit.statistic/Fit.dof
      
   else:
      a2 = model.bknpower.PhoIndx2.values[0]
      Ep = model.bknpower.BreakE.values[0]
      red_chi = Fit.statistic/Fit.dof
      
   a_1.append(a1)
   a_2.append(a2)
   e_p.append(Ep)
   indx.append(i)
   Red_chi.append(red_chi)
      
   parms = np.column_stack((indx,a_1,a_2,e_p,Red_chi))
   print("Photon Index1:", a1)
   print("Photon Index2:", a2)
   print(red_chi)


Spectrum #: 1 replaced 

1 spectrum  in use
 
Spectral Data File: /media/abhi/Data/737_spec/LPV/127/spec_grp_1.pi  Spectrum 1
Net count rate (cts/s) for Spectrum:1  5.048e-01 +/- 2.367e-02 (90.5 % total)
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-259
  Telescope: eROSITA Instrument: MERGED  Channel Type: PI
  Exposure Time: 996.8 sec
 Using fit statistic: chi
 Using Background File                ./em01_060153_020_BackgrSpec_00004_c010.fits
  Background Exposure Time: 996.8 sec
 Using Response (RMF) File            ./em01_060153_020_RMF_00004_c010.fits for Source 1
 Using Auxiliary Response (ARF) File  ./em01_060153_020_ARF_00004_c010.fits

Photon Index1: 7.0095180414568246
Photon Index2: 0
0.9393288337383776

Fit statistic  : Chi-Squared                976358.3     using 259 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 1

Test statistic : Chi-Squared                976358.3     using 259 bins.

**

In [19]:
id_LPV

array([127, 137, 294, 347, 427, 452, 544, 598, 643])

In [20]:
parms

array([[127.        ,   7.00951804,   0.        ,   0.        ,
          0.93932883],
       [137.        ,   7.83232956,   0.        ,   0.        ,
          1.05030695],
       [294.        ,   7.00737557,   0.        ,   0.        ,
          1.95875247],
       [347.        ,   5.85387375,   0.        ,   0.        ,
          1.51246935],
       [427.        ,   6.66276729,   0.        ,   0.        ,
          1.42012701],
       [452.        ,   7.2324915 ,   0.        ,   0.        ,
          1.74682338],
       [544.        ,   1.57858984,   0.        ,   0.        ,
          1.31156321],
       [598.        ,   1.76496027,   0.        ,   0.        ,
          0.99097422],
       [643.        ,   7.50041584,   0.        ,   0.        ,
          0.97670833]])

In [21]:
#Writing the parms in the text file
path = os.chdir("/home/abhi/Desktop/eROSITA/")
df = pd.DataFrame(parms)
df.to_csv("Vikas_LPV_parms.txt", index=False)